In [6]:
#Bibliotecas a importar
import numpy as np
import math as math
import itertools

### Sesión 4: Valuación de Opciones en tiempo discreto

In [11]:
#t_op: Tipo de opcion
# 1: Opción europea tipo call
# 2: Opción europea tipo put
# 3: Opción digital tipo call
# 4: Opción digital tipo put
# 5: Opción asiática
# 6: Opción lookback
t_op = 6
T = 1/4 #Tiempo de vigencia de la opción (en años)
r = 0.05 #tasa anual libre de riesgo
k = 4 #Precio strike. Funciona como el salto en el caso de las digitales
S_0 = 100 #Precio del subyacente a t=0
n = 3 #Particiones del árbol

u = 1.2 #Factor de aumento en el precio
d = 0.9 #Factor de disminución de precio

# Probabilidad en la medida libre de riesgo

In [12]:
delta_t = T / n
p = (math.exp(r * delta_t) - d) / (u - d) #Probabilidad de aumento en el precio
p

0.3472511976370615

# ESTO ES NOMAS PAL EJEMPLO X2

In [40]:
p=1/2

## Construcción del árbol

### Subyacente

Valor en el primer paso

In [13]:
#Matriz que almacena los valores del subyacente
arbol = np.zeros((n + 1, n + 1))
#El primer valor de la matriz es el subyacente a precio en t=0
arbol[0,0] = S_0
arbol #Visualziamos

array([[100.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

Valor del subyacente en el resto de los pasos

In [14]:
#For para llenar los valores de la matriz
for col in range(1, n +1):
    for ren in range(0, n +1):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            arbol[ren, col] = S_0 *(( u ** (col - ren)) * (d ** (ren)))
arbol #Visualizamos

array([[100. , 120. , 144. , 172.8],
       [  0. ,  90. , 108. , 129.6],
       [  0. ,   0. ,  81. ,  97.2],
       [  0. ,   0. ,   0. ,  72.9]])

### Payoff

Primer paso

In [15]:
#Matriz que almacena el valor de los payoffs
payoffs = np.zeros((n+1, n+1))
#Sobreescribimos la última columna de la matriz de PAYOFSS
#IMPORTANTE: Agregar condicional para el caso del put (este es el caso particular del call)
if t_op==1:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(arbol[ren, n] - k, 0)
elif t_op==2:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(k-arbol[ren, n] , 0)
elif t_op==3:
    for ren in range(0, n+1):
        if (arbol[ren, n] > S_0):
            payoffs[ren, n] = k
        else:
            base[ren, n] = 0
elif t_op==4:
    for ren in range(0, n+1):
        if (arbol[ren, n] < S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==5:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        suma=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            suma=suma+arbol[bajas,(i+1)]
        payoffs[ren,n]=max((suma/(n+1))-k,0)
elif t_op==6:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        payoffs[ren,n]=max(valores)
payoffs #Visualizamos

array([[  0. ,   0. ,   0. , 172.8],
       [  0. ,   0. ,   0. , 144. ],
       [  0. ,   0. ,   0. , 129.6],
       [  0. ,   0. ,   0. , 120. ],
       [  0. ,   0. ,   0. , 129.6],
       [  0. ,   0. ,   0. , 108. ],
       [  0. ,   0. ,   0. , 100. ],
       [  0. ,   0. ,   0. , 100. ]])

Valuamos backwards

In [16]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
if(t_op!=5):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1])

if(t_op==5 or t_op==6):
    for i in range(1, n+1):
        col = n - i
        for j in range(0, 2**col):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
    arbol_aux=np.zeros((2**n, n+1))
    for col in range(0,n+1):
        if(col==0):
            arbol_aux[0,0]=S_0
        else:
            up_down=list(itertools.product(['u','d'],repeat=col))
            for ren in range(0,2**col):
                bajas=0
                pos=up_down[ren]
                for letter in pos:
                    if (letter=='d'):
                        bajas=bajas+1
                arbol_aux[ren,col]=S_0*(u**(col-bajas))*d**(bajas)
payoffs #visualizamos

array([[113.88162075, 132.87095702, 153.36049931, 172.8       ],
       [  0.        , 104.50807145, 122.82079057, 144.        ],
       [  0.        ,   0.        , 115.02037448, 129.6       ],
       [  0.        ,   0.        ,  99.58420018, 120.        ],
       [  0.        ,   0.        ,   0.        , 129.6       ],
       [  0.        ,   0.        ,   0.        , 108.        ],
       [  0.        ,   0.        ,   0.        , 100.        ],
       [  0.        ,   0.        ,   0.        , 100.        ]])

In [17]:
arbol_aux

array([[100. , 120. , 144. , 172.8],
       [  0. ,  90. , 108. , 129.6],
       [  0. ,   0. , 108. , 129.6],
       [  0. ,   0. ,  81. ,  97.2],
       [  0. ,   0. ,   0. , 129.6],
       [  0. ,   0. ,   0. ,  97.2],
       [  0. ,   0. ,   0. ,  97.2],
       [  0. ,   0. ,   0. ,  72.9]])

# ESTO ES NOMAS PAL EJEMPLO

In [46]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
for i in range(1, n + 1):
    col = n - i
    for j in range(0, 2**col): 
        payoffs[j, col] = 5/16 * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
payoffs

array([[ 0.07247925,  0.45166016,  2.5       , 11.        ],
       [ 0.        ,  0.01220703,  0.390625  ,  5.        ],
       [ 0.        ,  0.        ,  0.078125  ,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

## Valor de la opción

In [18]:
payoffs[0,0]

113.88162074761169

## Composición del portafolio

alfas

In [19]:
#Matriz que almacena los valores de las alfas
alfas = np.zeros((n, n ))
if(t_op!=5):
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                alfas[ren,col]=(payoffs[ren,col+1]-payoffs[ren+1,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1])
if(t_op==5 or t_op==6):
    alfas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
            alfas[ren,col]=(payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1])
alfas #Visualizamos

array([[0.94542952, 0.84832524, 0.66666667],
       [0.        , 0.57171016, 0.2962963 ],
       [0.        , 0.        , 0.66666667],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

Betas

In [20]:
#Matriz que almacena los valores de las alfas
if(t_op!=5):
    betas = np.zeros((n, n ))
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[ren,col+1]-((payoffs[ren,col+1]-payoffs[ren+1,col+1])*arbol[ren,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1]))

if(t_op==5 or t_op==6):
    betas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[2*ren,col+1]-((payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])*arbol_aux[2*ren,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1]))
betas #Visualizamos

array([[19.33866885, 31.07192789, 57.36049931],
       [ 0.        , 53.05415713, 90.82079057],
       [ 0.        ,  0.        , 43.02037448],
       [ 0.        ,  0.        , 99.58420018],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

# Ignorar esto de abajo

In [3]:
pip- install itertools

ERROR: unknown command "-"
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
Note: you may need to restart the kernel to use updated packages.


In [5]:
import itertools

In [105]:
a=list(itertools.product(['u','d'],repeat=3))
a

[('u', 'u', 'u'),
 ('u', 'u', 'd'),
 ('u', 'd', 'u'),
 ('u', 'd', 'd'),
 ('d', 'u', 'u'),
 ('d', 'u', 'd'),
 ('d', 'd', 'u'),
 ('d', 'd', 'd')]

In [106]:
list(a[0])

['u', 'u', 'u']

In [107]:
a[0][0]

'u'

In [108]:
u=a[0]
u

('u', 'u', 'u')

In [110]:
u=u[:(len(u)-1)]
u

('u', 'u')

In [16]:
np.zeros((2,2**2))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [23]:
for i in range(0,2):
    print(i)

0
1


In [34]:
a=[[0,['u','d']]]

In [35]:
a

[[0, ['u', 'd']]]

In [9]:
up_down=list(itertools.product(['u','d'],repeat=1))
len(up_down[0])

1

In [3]:
print(6==5 or 6==6)

True
